In [32]:
from pwn import remote
from random import randint, choice
from Crypto.Util.number import long_to_bytes

In [33]:
def parse_flag(r):
    r.recvuntil(b"r = ")
    flag = eval(r.recvline().decode())
    r.recvuntil(b"s = ")
    s = eval(r.recvline().decode())
    r.recvuntil(b"t = ")
    t = eval(r.recvline().decode())
    return (flag, s, t)


def send_params(r, R, s, t):
    f = r.recvuntil(b"= ")
    r.sendline(f"{R}, {s}, {t}".encode())
    data = r.recvline().decode()
    if "decrypt" in data:
        R1 = int(r.recvline().decode()[3:].strip())
        s1 = int(r.recvline().decode()[3:].strip())
        t1 = int(r.recvline().decode()[3:].strip())
        return data, (R1, s1, t1)
    return data, (-1, -1, -1)


def send_random_params(r, B):
    R = randint(0, B)
    s = choice([-1, 1])
    t = choice([0, 1])
    data, ret = send_params(r, R, s, t)
    if ret[0] != R:
        return True, ret, R
    return False, None, None

def timing(r, a, b, coef = 0.006):
    while a <= b:
        m = (a + b) // 2
        s = 1
        t = 1
        start = time.time()
        d, _ = send_params(m, s, t)
        end = time.time()
        if end - start < coef:
            b = m - 1
        else:
            a = m + 1
        print(b - a, end - start)
    return a

In [52]:
def recover_n(r, B):
    samples1 = (0, 0)
    samples2 = (0, 0)
    
    while True:
        if len(samples2) * len(samples1) > 0 and is_prime(samples1[0]) and is_prime(samples2[0]):
            if samples1[0] == samples2[0]:
                return None
            break
        
        flag, ret, R = send_random_params(r, B)
        if flag:
            print("here")
              
            rs = set()
            for s, t in ((-1, 1), (-1, 0), (1, 1), (1, 0)):
                data, ret = send_params(r, R, s, t)
                rs.add(ret[0])
            
            rs = list(rs)
            if len(rs) == 2:
                print("PIZDA")
                
                tmp = gcd(R - rs[0], R - rs[1])
                
                
                if samples1[0] == 0 and samples2[0] == 0:
                    samples1 = (tmp, rs, R)
                    print("initiated samples for q")
                    continue       
                    
                tmp1 = gcd(tmp, samples1[0])
                if tmp1 > 2**1022:
                    samples1 = (tmp1, rs, R)
                    #print(samples1)
                    continue
                
                if samples2[0] == 0:
                    samples2 = (tmp, rs, R)
                    print("initiated samples for p")
                    continue
                
                tmp2 = gcd(tmp, samples2[0])
                if tmp2 > 2**1022:
                    samples2 = (tmp2, rs, R)   
                    #print(samples2)
                    
    return samples1, samples2  

In [68]:
#r = remote("localhost", int(17778))
r = remote("34.141.16.87", int(50001))
rf, sf, tf = parse_flag(r)
from cur import p as p1, q as q1, n as n1, c as c1
    
while True:
    samples = recover_n(r, 100000000)
    if samples is not None:
        break
        
q = samples[0][0]
p = samples[1][0]
n = p * q
   
print("Found n")
    
print("Found q")
r1, r2 = samples[0][1]
R = samples[0][2]
assert gcd(r1 - R, q) == q
   
lp = ((r2 * R - R**2) * pow(-2*R + r1 + r2, -1, p)) % p
cp = (r1 * lp - lp**2) % p
#assert cp == c1 % p

r1, r2 = samples[1][1]
R = samples[1][2]
lq = ((r2 * R - R**2) * pow(-2 * R + r1 + r2, -1, q)) % q
cq = (r1 * lq - lq**2) % q
#assert cq == c1 % q
print("Found cp")

c = crt([int(cp), int(cq)], [p, q])

x = var('x')
rp = PolynomialRing(GF(p), x)(x**2 - rf * x + c).roots()
rq = PolynomialRing(GF(q), x)(x**2 - rf * x + c).roots()

for r1, _ in rp:
    for r2, _ in rq:
        root = crt([int(r1), int(r2)], [p, q])
        print(long_to_bytes(int(root)))

[x] Opening connection to 34.141.16.87 on port 50001
[x] Opening connection to 34.141.16.87 on port 50001: Trying 34.141.16.87
[+] Opening connection to 34.141.16.87 on port 50001: Done
here
PIZDA
initiated samples for q
here
PIZDA
here
PIZDA
here
PIZDA
here
here
PIZDA
initiated samples for p
Found n
Found q
Found cp
b'U\x85\'\x9b\xe3g\x17\x04\x95k}\x1dXt|Z\xfc\xefp\xaf;{<\xdd\xe5\x1d\x9b)\xa3\xa1j2\xee\x97\xc9\x83\xfcF[\x11.\x9b\xfc@uL\x84\xf5\xbe\x0e,\x862\xd6\x12_\xbe\x9f=\xebUe\xc4\'\xbeuF\xc5G\xdd\xbf\xfe\xfd\x92\xd8\xa6\xe7\x12\xafG\xed~\xc6"\xa2\x81\x93\xe8s\xf5\xda\x81\x9f\xd26\xcf\x90\xaf\xa8`;\xd4\xdd9Hn\xfc\xc5\xb4En\xbe\xa1G\xaatv\xbcm.\xad\xd1\xe0,\xaa\xb7\xb0N\x90\xae M\x85V<\xbd\x84\xb8k!\x1c\xac\xb5>\xfe\xc2\x13%\xf3\x8a`\x07\xd0\xce\xf6<\x9f\xc8{\xe9\x87\xb8?\xa5\x97\x01`]g\xfc:\xa4\x81KW\xf7\x14\xf1\xb9\x19\x9cl\xa6)s\xc8\xc7\xf3X\xdf8\x86\xf5h\x86\x13\x80\xef\xdd\x94\xf1Yo;\x94\xfd\x96;\xcb\x9a\xf7\xb9\xe7{\x9e\x92\xdd\xf5#\xa8\x8c\x16\xb9\x0e\xf7\xf3\xfda\xc9\xa926\

In [61]:
q, rs, R = samples[0]

In [62]:
gcd(rs[0] - R, q)

173796213260871170040700220192655758173876810603482384221006789178676837014031666781857032734198923047075619236140558295903138749318494699363904097824001464204821533648947910209450585219200170463882208186045461234435411625066873291773686810755321693249823632684762242005167031472798143659195785148736182106643

In [65]:
(-2 * R + rs[0] + rs[1]) % p

57042301188525303182251061758966075354475613252687016840453340828658988155302505247778388303808042726198259423062963147499285497066003820003830337708261246154162121844840482389274854353450052699911369312735539683525796764932194611851320674687807410796534319565960251826493648702331572210187276311484201109848